In [3]:
! pip install kaggle

     -------------------------------------- 82.1/82.1 kB 764.4 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached certifi-2024.6.2-py3-none-any.whl (164 kB)
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
     -------------------------------------- 162.8/162.8 kB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 78.2/78.2 kB 4.5 MB/s eta 0:00:00


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python311\\Scripts\\slugify.exe' -> 'C:\\Python311\\Scripts\\slugify.exe.deleteme'


[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Uploading the kaggle.json

In [1]:
# configuring the path of kaggle.json file
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

The syntax of the command is incorrect.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.


Importing Twitter Sentiment Dataset

In [2]:
# API to fetch the Dataset from Kaggle
! kaggle datasets download -d kazanova/sentiment140

'kaggle' is not recognized as an internal or external command,
operable program or batch file.


Extracting the Dataset

In [6]:
# extracting the dataset
from zipfile import ZipFile
dataset = "/content/sentiment140.zip"

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print("the dataset has been extracted")

the dataset has been extracted


Importing the Dependencies

In [7]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk. stem. porter import PorterStemmer
from sklearn.feature_extraction. text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
#printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [10]:
# from dataset to pandas dataframe
twitter_data= pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')

In [11]:
# checking the number of rows and columns
twitter_data.shape

(1599999, 6)

In [12]:
# printing first 5 rows
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [13]:
# naming the columns and reading the dataset again
column_names = ['target', 'date', 'flag', 'user', 'text' ]
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names=column_names, encoding = 'ISO-8859-1')

In [14]:
# checking the number of rows and columns
twitter_data.shape

(1600000, 5)

In [15]:
# printing first 5 rows
twitter_data.head()

,target,date,flag,user,text
0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [16]:
# counting the number of missing values
twitter_data.isnull().sum()

target    0
date      0
flag      0
user      0
text      0
dtype: int64

In [17]:
# checking the distribution of the target columns
twitter_data['target'].value_counts()

target
2190457769    2
1972193428    2
1989776729    2
1989776908    2
1564543229    2
             ..
2197311196    1
2197311146    1
2197310899    1
2197310477    1
2193602129    1
Name: count, Length: 1598315, dtype: int64

Converting the target 4 to 1

In [18]:
twitter_data.replace({ 'target' :{4:1}}, inplace=True)

In [19]:
# checking the distribution of the target columns
twitter_data['target'].value_counts()

target
2190457769    2
1972193428    2
1989776729    2
1989776908    2
1564543229    2
             ..
2197311196    1
2197311146    1
2197310899    1
2197310477    1
2193602129    1
Name: count, Length: 1598315, dtype: int64

0 --> Negative Tweet

1 --> Positive Tweet

## Stemming

Stemming is the processs of reducing a word to its root word

In [20]:
port_stem = PorterStemmer()

In [21]:
def stemming(content):
    # Initialize PorterStemmer
    port_stem = PorterStemmer()

    # Remove non-alphabet characters and convert to lower case
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content).lower()

    # Split the content into words
    stemmed_content = stemmed_content.split()

    # Remove stopwords and apply stemming
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if word not in stopwords.words('english')]

    # Join the words back into a single string
    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content

In [ ]:
twitter_data['stemmed_content']=twitter_data['text'].apply(stemming)

In [ ]:
twitter_data.head()

In [ ]:
print(twitter_data['stemmed_content'])

In [ ]:
print(twitter_data['target'])

In [ ]:
twitter_data.replace({'target':{4:1}}, inplace=True)

In [ ]:
print(twitter_data['target'])

In [ ]:
# separating the data and label
X  = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [ ]:
print(X)

In [ ]:
print(Y)

In [ ]:
#  Splitting the data to training data and test data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

In [ ]:
print(X_train)

In [ ]:
print(X_test)

In [ ]:
# converting the textual data to numerical data

vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
print(X_train)

In [ ]:
print(X_test)

Training the Machine Learning Model

Logistic Regression

In [ ]:
model = LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train, Y_train)

Accuracy

In [ ]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [ ]:
print('the accuracy score on training data is: ', training_data_accuracy)